<a href="https://colab.research.google.com/github/leoclementliao/bert-cls-notebook/blob/master/FinalVersion_S2_pseudo_label.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!/opt/bin/nvidia-smi

Sun Apr 26 08:25:08 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   28C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
version_name = 'FV_Bert'
test_name = f'/content/drive/My Drive/Colab Notebooks/kaggle_covid19NLP/submission/test_sub{version_name}.csv'
lr_global = 1e-6
lr_dense = 2e-4
batch_size = 80
epochs_num = 3

In [4]:
% cp -r /content/drive/My\ Drive/Colab\ Notebooks/kaggle_covid19NLP /content
% cd /content/kaggle_covid19NLP/notebook
# ! unzip /content/kaggle_covid19NLP/RoBERTa_zh_L12_PyTorch.zip -d /content/kaggle_covid19NLP/RoBERTa_zh_L12_PyTorch
# ! unzip /content/kaggle_covid19NLP/data/test_dataset.zip -d /content/kaggle_covid19NLP/data/test_dataset

/content/kaggle_covid19NLP/notebook


# Install transformers library

In [5]:
!pip install transformers

     |████████████████████████████████| 573kB 4.9MB/s 
     |████████████████████████████████| 3.7MB 59.9MB/s 
     |████████████████████████████████| 1.0MB 53.1MB/s 
     |████████████████████████████████| 890kB 48.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=830ae6ff077d127528d6fe801dd34fd9e8d84b5fbaaffb53cc0eebc04c94d0fb
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_probability as tfp


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
# import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint

# from tensorflow.keras.layers import (GlobalAveragePooling1D, Reshape, Conv1D,
#                                      multiply)
import os
import time, gc

from transformers import *
print(tf.__version__)
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)


2.2.0-rc3


In [8]:
UNLABEL_PATH = '../data/train_dataset/nCoV_900k_train.unlabled.csv' 
SUB_PATH = '../data/submit_example.csv'

MAX_SEQUENCE_LENGTH = 140
input_categories = '微博中文内容'
output_categories = '情感倾向'

df_test = pd.read_csv(UNLABEL_PATH)
df_sub = pd.read_csv(SUB_PATH)

print('unlabeled data shape =', df_test.shape)
df_test.head(5)

unlabeled data shape = (900000, 6)


,微博id,微博发布时间,发布人账号,微博中文内容,微博图片,微博视频
0,4456074364642450,01月01日 23:59,Dear_Christin,2020年第一天，宝宝晚上发烧了。上个月的肺炎经历还历历在目，不觉又开始揪心。目前低烧安稳睡...,[],[]
1,4456069114155340,01月01日 23:38,Maggie大美Y,生病发烧一直就没好～就当预示着2020年是个风风火火的好好好运年～新年第一天有肉吃，以后天天...,['https://ww1.sinaimg.cn/orj360/006TMJgaly1gah...,[]
2,4456068145061890,01月01日 23:34,折眉几年,害新年第一天就发烧????,[],[]
3,4456064403586410,01月01日 23:20,秃头少女Brushblog,是心动啊，糟糕眼神躲不掉。对你莫名的心跳，竟然停不了对你的迷恋感觉要发烧。@namjoohy...,['https://wx3.sinaimg.cn/orj480/63becc16gy1g7v...,['https://f.video.weibocdn.com/0009osQVlx07xJg...
4,4456060284446840,01月01日 23:03,Akemi庄蝶,过去的一整年有好好喜欢你。不过今年不可以啦，毕竟，可不能连自己都不要了。跨年给你打电话，听到...,[],[]


# Convert tokens to input_ids
<img src="https://imgkr.cn-bj.ufileos.com/341d1c83-45cf-4e9b-a656-a547cd0f2c67.png" width="50%" height="50%" />

transformer chinese weight
- https://github.com/ymcui/Chinese-BERT-wwm
- https://github.com/brightmart/roberta_zh



In [0]:
def _convert_to_transformer_inputs(instance, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for transformer (including bert)"""
    """ return input_ids,token_type_ids,attention_mask"""
    inputs = tokenizer.encode_plus(instance,
        add_special_tokens=True,
        max_length=max_sequence_length,
        truncation_strategy= 'longest_first')

    input_ids =  inputs["input_ids"]
    input_masks = inputs["attention_mask"]
    input_segments = inputs["token_type_ids"]
    padding_length = max_sequence_length - len(input_ids)
    #padding
    padding_id = tokenizer.pad_token_id
    input_ids = input_ids + ([padding_id] * padding_length)
    input_masks = input_masks + ([0] * padding_length)
    input_segments = input_segments + ([0] * padding_length)
    return [input_ids, input_masks, input_segments]


def compute_input_arrays(df, columns, tokenizer, max_sequence_length):
    input_ids, input_masks, input_segments = [], [], []
    for instance in tqdm(df[columns]):
        
        ids, masks, segments = _convert_to_transformer_inputs(str(instance), tokenizer, max_sequence_length)
        
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)

    return [np.asarray(input_ids, dtype=np.int32), 
            np.asarray(input_masks, dtype=np.int32), 
            np.asarray(input_segments, dtype=np.int32)
           ]

In [10]:
BERT_PATH = '/content/kaggle_covid19NLP/bert_base_chinese'
tokenizer = BertTokenizer.from_pretrained(BERT_PATH+'/bert-base-chinese-vocab.txt')

test_inputs = compute_input_arrays(df_test, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)

Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated
100%|██████████| 900000/900000 [08:38<00:00, 1736.46it/s]


# BERT model

https://huggingface.co/models


<img src="https://imgkr.cn-bj.ufileos.com/9115ac01-f455-498b-8c38-9c4abb04046c.png" width="50%" height="50%" />




In [0]:
def f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [0]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer
class Initi(tf.keras.initializers.Initializer ):

    """Initializer that generates tensors initialized to 1.

    """



    def __call__(self, shape, dtype=None):

        out = np.zeros( shape=shape, dtype=np.float32)
        #out[0] = 1
        #out[-1] = -1
        #out[-2] = 0
        #out[-3] = 0
        #out[-4] = 0
        return out


class Weighted_sum(Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(Weighted_sum, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='weighted_sum', 
                                      shape=(input_shape[-1], self.output_dim),
                                      initializer=Initi(),
                                      trainable=True)
        super(Weighted_sum, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        return K.dot(x, K.softmax(self.kernel,-2))#tf.linalg.matmul

    def compute_output_shape(self, input_shape):
        return input_shape#(input_shape[0], self.output_dim)


def SE_layer(stacked_layers):
    num_filters_out = stacked_layers.shape[2]
    se = tf.keras.layers.GlobalAveragePooling1D()(stacked_layers)
    se = tf.keras.layers.Reshape((1, num_filters_out))(se)
    se = tf.keras.layers.Conv1D(num_filters_out//4, 1,
                padding='same',
                activation="relu",
                kernel_initializer='he_normal',
                name='SE_layer1')(se)
    se = tf.keras.layers.Conv1D(num_filters_out, 1,
                padding='same',
                activation='sigmoid',
                kernel_initializer='he_normal',
                name='SE_layer2')(se)

    Layers_SE = tf.keras.layers.multiply([stacked_layers, se])
    return Layers_SE

In [0]:
#from tensorflow.keras.legacy import interfaces
import tensorflow.keras.backend as K
from tensorflow.python.keras.optimizers import Optimizer

class Adam_lr_mult(Optimizer):
    """Adam optimizer.
    Adam optimizer, with learning rate multipliers built on Keras implementation
    # Arguments
        lr: float >= 0. Learning rate.
        beta_1: float, 0 < beta < 1. Generally close to 1.
        beta_2: float, 0 < beta < 1. Generally close to 1.
        epsilon: float >= 0. Fuzz factor. If `None`, defaults to `K.epsilon()`.
        decay: float >= 0. Learning rate decay over each update.
        amsgrad: boolean. Whether to apply the AMSGrad variant of this
            algorithm from the paper "On the Convergence of Adam and
            Beyond".
    # References
        - [Adam - A Method for Stochastic Optimization](http://arxiv.org/abs/1412.6980v8)
        - [On the Convergence of Adam and Beyond](https://openreview.net/forum?id=ryQu7f-RZ)
        
    AUTHOR: Erik Brorson
    """

    def __init__(self, lr=0.001,name = 'mAdam', beta_1=0.9, beta_2=0.999,
                 epsilon=None, decay=0., amsgrad=False,
                 multipliers=None, debug_verbose=False,**kwargs):
        super(Adam_lr_mult, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.lr = K.variable(lr, name='lr')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
        if epsilon is None:
            epsilon = K.epsilon()
        self.epsilon = epsilon
        self.initial_decay = decay
        self.amsgrad = amsgrad
        self.multipliers = multipliers
        self.debug_verbose = debug_verbose

    #@interfaces.legacy_get_updates_support
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        lr = self.lr
        if self.initial_decay > 0:
            lr *= (1. / (1. + self.decay * K.cast(self.iterations,
                                                  K.dtype(self.decay))))

        t = K.cast(self.iterations, K.floatx()) + 1
        lr_t = lr * (K.sqrt(1. - K.pow(self.beta_2, t)) /
                     (1. - K.pow(self.beta_1, t)))

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        if self.amsgrad:
            vhats = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        else:
            vhats = [K.zeros(1) for _ in params]
        self.weights = [self.iterations] + ms + vs + vhats

        for p, g, m, v, vhat in zip(params, grads, ms, vs, vhats):

            # Learning rate multipliers
            if self.multipliers:
                multiplier = [mult for mult in self.multipliers if mult in p.name]
            else:
                multiplier = None
            if multiplier:
                new_lr_t = lr_t * self.multipliers[multiplier[0]]
                if self.debug_verbose:
                    print('Setting {} to learning rate {}'.format(multiplier[0], new_lr_t))
                    print(K.get_value(new_lr_t))
            else:
                new_lr_t = lr_t
                if self.debug_verbose:
                    print('No change in learning rate {}'.format(p.name))
                    print(K.get_value(new_lr_t))
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)
            if self.amsgrad:
                vhat_t = K.maximum(vhat, v_t)
                p_t = p - new_lr_t * m_t / (K.sqrt(vhat_t) + self.epsilon)
                self.updates.append(K.update(vhat, vhat_t))
            else:
                p_t = p - new_lr_t * m_t / (K.sqrt(v_t) + self.epsilon)

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {'lr': float(K.get_value(self.lr)),
                  'beta_1': float(K.get_value(self.beta_1)),
                  'beta_2': float(K.get_value(self.beta_2)),
                  'decay': float(K.get_value(self.decay)),
                  'epsilon': self.epsilon,
                  'amsgrad': self.amsgrad,
                  'multipliers':self.multipliers}
        base_config = super(Adam_lr_mult, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


In [0]:
learning_rate_multipliers = {}
learning_rate_multipliers['weighted_sum'] = lr_dense/lr_global
learning_rate_multipliers['dense0'] = lr_dense/lr_global
learning_rate_multipliers['dense1'] = lr_dense/lr_global
learning_rate_multipliers['dense2'] = lr_dense/lr_global
#learning_rate_multipliers['layer_3'] = 0.1


In [0]:
from tensorflow.python.framework.ops import disable_eager_execution

disable_eager_execution()

In [0]:
def create_model():
    input_id = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)    
    input_mask = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)    
    input_atn = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)    
    config = BertConfig.from_pretrained( '../bert_base_chinese/bert-base-chinese-config.json', output_hidden_states=True)
    bert_model = TFBertModel.from_pretrained('../bert_base_chinese/bert-base-chinese-tf_model.h5', config=config)
    # config = BertConfig.from_pretrained( '../RoBERTa_zh_L12_PyTorch/config.json', output_hidden_states=True)
    # bert_model = TFBertModel.from_pretrained('../RoBERTa_zh_L12_PyTorch/pytorch_model.bin',from_pt=True, config=config)
    
    sequence_output, pooler_output, hidden_states = bert_model(input_id, attention_mask=input_mask, token_type_ids=input_atn)
    print(sequence_output.shape)
    print(len(hidden_states))
    #print(hidden_states[-1].shape)
    #(bs,140,768)(bs,768)
    
    stacked_layers = tf.stack([tf.keras.layers.Dropout(0.1)(layer[:, 0, :]) for layer in hidden_states[1:]],axis = -1)#,noise_shape=(None,None,1)
    
    #print(shape)
    # stacked_layers = SE_layer(stacked_layers)
    cls_output = Weighted_sum(1)(stacked_layers)
    cls_output = K.squeeze(cls_output,axis = -1)
    print(cls_output.shape)
    #cls_output = tf.keras.layers.GlobalAveragePooling1D()(cls_output)
    #x = tf.keras.layers.Dropout(0.2)(x)
    #x = tf.keras.layers.Dense(3, activation='softmax',name = "out")(x)
    # multisample dropout (wut): https://arxiv.org/abs/1905.09788
    dense_out = []
    FC0  = tf.keras.layers.Dense(128, activation='relu',name = "dense0")
    FC1 = tf.keras.layers.Dense(128, activation='relu',name = "dense1")
    FC2  = tf.keras.layers.Dense(3, activation='softmax',name = "dense2")
    for _ in range(8):
        x = tf.keras.layers.Dropout(0.15)(cls_output)
        
        x = FC0(x)
        x = tf.keras.layers.Dropout(0.15)(x)
        
        #x = FC1(x)
        #x = tf.keras.layers.Dropout(0.2)(x)
        x = FC2(x)
        dense_out.append(x)
    
    out = tf.keras.layers.average(dense_out)

    model = tf.keras.models.Model(inputs=[input_id, input_mask, input_atn], outputs=out)
    #layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])
    bert_model.bert.pooler.dense.trainable  = False
    #optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
    opt = Adam_lr_mult(lr = lr_global, multipliers=learning_rate_multipliers)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['acc', f1])

    # FL=lambda y_true,y_pred: Focal_Loss(y_true, y_pred, alpha=0.25, gamma=2)
    # model.compile(loss=FL, optimizer=opt, metrics=['acc', f1])   
    
    return model

In [17]:
model = create_model()
# model.save_weights("init_weights.h5")
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
(None, 140, 768)
13
(None, 768)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 140)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 140)]        0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 140)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 140, 768), ( 102267648   input_1[0][0]  

In [0]:
# import glob
# model_list = glob.glob('/content/kaggle_covid19NLP/model/FV_*.h5')
model_list=['/content/kaggle_covid19NLP/model/bert_drop-0.h5',
            '/content/kaggle_covid19NLP/model/bert_drop-1.h5',
            '/content/kaggle_covid19NLP/model/bert_drop-2.h5',
            '/content/kaggle_covid19NLP/model/bert_drop-3.h5',
            '/content/kaggle_covid19NLP/model/bert_drop-4.h5']

In [19]:
from datetime import datetime
print("now =", datetime.now() )

now = 2020-04-26 08:38:14.641143


In [20]:
df_pdata = pd.DataFrame(columns=['微博id', input_categories, '-1', '0', '1'])
df_pdata['微博id'] = df_test['微博id']
df_pdata[input_categories] = df_test[input_categories]

test_preds = []
start_time = time.time()
for i, model_path in enumerate(model_list):
    model.load_weights(model_path)
    print(model_path)
    test = model.predict(test_inputs, batch_size = 2048)
    df_pdata[['-1', '0', '1']] = test

    df_pdata.to_csv(f'/content/drive/My Drive/Colab Notebooks/kaggle_covid19NLP/data/train_dataset/nCoV_900k_train.Plabled{model_path[-4]}.csv', index=False)
    test_preds.append(test)
    
    print(time.time()-start_time)
    start_time = time.time()

pseudo_label = np.average(test_preds, axis=0)

/content/kaggle_covid19NLP/model/bert_drop-0.h5
4198.5248103141785
/content/kaggle_covid19NLP/model/bert_drop-1.h5
4187.188954114914
/content/kaggle_covid19NLP/model/bert_drop-2.h5
4187.199415683746
/content/kaggle_covid19NLP/model/bert_drop-3.h5
4187.453500509262
/content/kaggle_covid19NLP/model/bert_drop-4.h5
4187.298005104065


In [0]:
pseudo_label = np.average(test_preds, axis=0)
df_pdata[['-1', '0', '1']] = pseudo_label
df_pdata.to_csv('/content/drive/My Drive/Colab Notebooks/kaggle_covid19NLP/data/train_dataset/nCoV_900k_train.Plabled.csv', index=False)

In [31]:
df_good = df_pdata[(df_pdata['-1']>0.6) | (df_pdata['0']>0.6) | (df_pdata['1']>0.6)]
df_good.head()

,微博id,微博中文内容,-1,0,1
2,4456068145061890,害新年第一天就发烧????,0.945920,0.051564,0.002515
5,4456059202748560,2020请对我好点，第一天就感冒发烧…………?,0.764661,0.221836,0.013503
7,4456071538097160,2020的第一天是这样的感冒发烧流鼻涕2厦门?,0.817297,0.178157,0.004546
8,4456071366149900,我发烧我胃痛你在乎吗我通宵我失踪你在乎吗我难过我不安你在乎吗我委屈我心痛你在乎吗，我爱你你在乎?,0.826483,0.150577,0.022940
9,4456070334552490,20.1.1发烧，一个晚上没睡好，早上起来就去诊所看了一下，晚上写作业，明天又有考试?,0.793562,0.201865,0.004573


In [0]:
pred_cat = np.argmax(df_good[['-1','0','1']].values, axis=1)

In [37]:
num_total = len(pred_cat)
print('-1 ration', (pred_cat==0).sum()/num_total)
print('0 ration', (pred_cat==1).sum()/num_total)
print('1 ration', (pred_cat==2).sum()/num_total)



-1 ration 0.15889685546492033
0 ration 0.5935058007917213
1 ration 0.24759734374335832
